<a href="https://colab.research.google.com/github/johngeoscrub/geoscrub-google-collab-tutorials/blob/main/Quick_Geotiff_Slicing_Recipe_for_AI_and_Machine_Learning_Applications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quick GeoTIFF Slicing Recipe for Computer Vision

This notebook accompanies the article "Quick GeoTIFF Slicing Recipe for Computer Vision", providing practical guidance on preparing GeoTIFF images for computer vision applications. We'll cover how to slice large GeoTIFF files into 256x256 pixel tiles and address issues with residual pieces through padding.

## Introduction

In this notebook, we demonstrate techniques for efficiently processing GeoTIFF files, making them suitable for computer vision models. These methods are crucial for handling large and complex geospatial imagery by breaking them down into manageable pieces.

## Setup

First, ensure GDAL is installed in your Colab environment. Run the following command:

```bash
!pip install gdal


#### Loading GeoTIFF Data
Let's start by loading a GeoTIFF file using GDAL. This step is foundational for understanding how to work with and manipulate geospatial imagery.

In [ ]:
from osgeo import gdal
import os

# Constants
INPUT_PATH = "***Path to Geotiff***"  # Ensure this path points to your GeoTIFF file
OUTPUT_DIR = "***Path to Output Directory***"  # Specify the directory where tiles will be saved
TILE_SIZE = 640 # Define the tile size in pixels

# Load our Dataset
dataset = gdal.Open(INPUT_PATH)
if dataset is None:
    raise IOError("Could not open the GeoTIFF file.")

#### Slicing GeoTIFF into Manageable Tiles
Next, we'll slice the GeoTIFF into 640x640 pixel tiles, ensuring they are suitable for computer vision tasks. The following function facilitates this process:

In [ ]:
# Extract file name
image_file_name = os.path.splitext(os.path.basename(INPUT_PATH))[0]

# Get the size of the GeoTIFF
width, height = dataset.RasterXSize, dataset.RasterYSize

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Iterate through and slice our dataset vertically and horizontally by TILE_SIZE
for xoff in range(0, width, TILE_SIZE):
    for yoff in range(0, height, TILE_SIZE):
        # Calculate the size of the current tile
        xsize, ysize = min(TILE_SIZE, width - xoff), min(TILE_SIZE, height - yoff)

        # Generate output file name
        output_filename = os.path.join(
            OUTPUT_DIR,
            f"{image_file_name}_{xoff}_{yoff}_{xsize}x{ysize}.tif"
        )

        # Use gdal_translate to extract the tile
        gdal.Translate(output_filename, dataset, srcWin=[xoff, yoff, xsize, ysize])

#### Handling Residual Tiles with Padding
Since our GeoTIFF might not divide evenly into 640x640 tiles, we'll use the following function to pad any residual tiles. This ensures all tiles are of uniform size without distorting the original data.

In [ ]:
def enforce_image_dimensions_black_fill(input_slice_tif: str, enforced_dimension: int) -> None:
    """
    Resize a GeoTIFF image to a specified dimension, filling any extra space with black pixels.
    The original image is centered in the new dimensions, and its georeferencing is preserved.

    Args:
    input_slice_tif (str): The file path of the original GeoTIFF image.
    enforced_dimension (int): The minimum width and height for the new image dimensions.
    """
    src_ds = gdal.Open(input_slice_tif)

    # Get original dimensions
    orig_width = src_ds.RasterXSize
    orig_height = src_ds.RasterYSize

    if orig_height != enforced_dimension or orig_width != enforced_dimension:

      # Get original georeferencing and projection
      geotransform = src_ds.GetGeoTransform()
      projection = src_ds.GetProjection()

      # Calculate new dimensions and offsets to center the original image
      new_width = max(orig_width, enforced_dimension)
      new_height = max(orig_height, enforced_dimension)
      x_offset = (new_width - orig_width) // 2
      y_offset = (new_height - orig_height) // 2

      # Create new GeoTIFF
      driver = gdal.GetDriverByName('GTiff')
      output_file_path = input_slice_tif
      os.remove(input_slice_tif)
      dst_ds = driver.Create(output_file_path, new_width, new_height, src_ds.RasterCount, src_ds.GetRasterBand(1).DataType)

      # Fill new GeoTIFF with black
      for i in range(dst_ds.RasterCount):
          dst_ds.GetRasterBand(i + 1).Fill(0)  # Fill with black

      # Copy original data into new GeoTIFF at calculated position
      dst_ds.WriteRaster(x_offset, y_offset, orig_width, orig_height, src_ds.ReadRaster())

      # Update the geotransform to reflect the new positioning
      new_geotransform = list(geotransform)
      new_geotransform[0] -= geotransform[1] * x_offset  # Adjusting X-origin
      new_geotransform[3] -= geotransform[5] * y_offset  # Adjusting Y-origin

      # Set the updated georeferencing and projection
      dst_ds.SetGeoTransform(new_geotransform)
      dst_ds.SetProjection(projection)

      # Close datasets
      src_ds = None
      dst_ds = None

#### Example Usage
To see our functions in action, let's apply them to a sample GeoTIFF file:

In [ ]:
# Iterate all GeoTiffs in our OUTPUT_DIR
for filename in os.listdir(OUTPUT_DIR):
  if filename.endswith(".tif"):
    enforce_image_dimensions_black_fill(os.path.join(OUTPUT_DIR, filename), TILE_SIZE)